In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report

In [8]:
# Load dataset
dataset_path = "C:\\Users\\ASUS\\Desktop\\support ticker\\dataset.csv"
df = pd.read_csv(dataset_path)

In [9]:
df.head()

Ticket ID        Customer Name              Customer Email  Customer Age  \
0          1        Marisa Obrien  carrollallison@example.com            32   
1          2         Jessica Rios    clarkeashley@example.com            42   
2          3  Christopher Robbins   gonzalestracy@example.com            48   
3          4     Christina Dillon    bradleyolson@example.org            27   
4          5    Alexander Carroll     bradleymark@example.com            67   

  Customer Gender  ProductPurchased Date of Purchase      Ticket Type  \
0           Other        GoPro Hero        3/22/2021  Technical issue   
1          Female       LG Smart TV        5/22/2021  Technical issue   
2           Other          Dell XPS        7/14/2020  Technical issue   
3          Female  Microsoft Office       11/13/2020  Billing inquiry   
4          Female  Autodesk AutoCAD         2/4/2020  Billing inquiry   

             Ticket Subject  \
0             Product setup   
1  Peripheral compatibility   
2           Network problem   
3            Account access   
4                 Data loss   

                                  Ticket Description  \
0  I'm having an issue with the {product_purchase...   
1  I'm having an issue with the {product_purchase...   
2  I'm facing a problem with my {product_purchase...   
3  I'm having an issue with the {product_purchase...   
4  I'm having an issue with the {product_purchase...   

               Ticket Status                                     Resolution  \
0  Pending Customer Response                                            NaN   
1  Pending Customer Response                                            NaN   
2                     Closed   Case maybe show recently my computer follow.   
3                     Closed  Try capital clearly never color toward story.   
4                     Closed                    West decision evidence bit.   

  Ticket Priority Ticket Channel First Response Time Time to Resolution  \
0        Critical   Social media      6/1/2023 12:15                NaN   
1        Critical           Chat      6/1/2023 16:45                NaN   
2             Low   Social media      6/1/2023 11:14     6/1/2023 18:05   
3             Low   Social media       6/1/2023 7:29      6/1/2023 1:57   
4             Low          Email       6/1/2023 0:12     6/1/2023 19:53   

   Customer Satisfaction Rating  
0                           NaN  
1                           NaN  
2                           3.0  
3                           3.0  
4                           1.0

In [10]:
# Print the column names to check for any discrepancies
print(df.columns)

Index(['Ticket ID', 'Customer Name', 'Customer Email', 'Customer Age',
       'Customer Gender', 'ProductPurchased', 'Date of Purchase',
       'Ticket Type', 'Ticket Subject', 'Ticket Description', 'Ticket Status',
       'Resolution', 'Ticket Priority', 'Ticket Channel',
       'First Response Time', 'Time to Resolution',
       'Customer Satisfaction Rating'],
      dtype='object')


In [11]:
df = df[['ProductPurchased','Ticket Description', 'Ticket Type','Ticket Subject']]

In [12]:
# Combine 'Ticket Subject' and 'Ticket Description' for text input
df['text'] = df['Ticket Subject'] + " " + df['Ticket Description'] + " " + df['ProductPurchased']

In [13]:
# Encode labels (Ticket Type)
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['Ticket Type'])


In [14]:
# Tokenize text data
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(df['text'])
X = tokenizer.texts_to_sequences(df['text'])
X = pad_sequences(X, padding='post')

In [15]:
# Get labels
y = df['label'].values

In [57]:
# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [59]:
# Build the RNN model
model = Sequential()

# Embedding layer to convert words into dense vectors
model.add(Embedding(input_dim=5000, output_dim=128, input_length=X_train.shape[1]))

# LSTM layer
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))

# Fully connected output layer with softmax activation
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(len(label_encoder.classes_), activation='softmax'))

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=64, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test accuracy: {accuracy}")

# Save the model
model.save('ticket_inquiry_rnn_model.h5')


Epoch 1/100
106/106 ━━━━━━━━━━━━━━━━━━━━ 8s 51ms/step - accuracy: 0.1974 - loss: 1.6115 - val_accuracy: 0.2037 - val_loss: 1.6107
Epoch 2/100
106/106 ━━━━━━━━━━━━━━━━━━━━ 5s 47ms/step - accuracy: 0.2104 - loss: 1.6097 - val_accuracy: 0.2078 - val_loss: 1.6107
Epoch 3/100
106/106 ━━━━━━━━━━━━━━━━━━━━ 5s 47ms/step - accuracy: 0.1990 - loss: 1.6091 - val_accuracy: 0.2031 - val_loss: 1.6098
Epoch 4/100
106/106 ━━━━━━━━━━━━━━━━━━━━ 5s 47ms/step - accuracy: 0.2127 - loss: 1.6098 - val_accuracy: 0.2084 - val_loss: 1.6097
Epoch 5/100
106/106 ━━━━━━━━━━━━━━━━━━━━ 5s 48ms/step - accuracy: 0.2089 - loss: 1.6089 - val_accuracy: 0.1907 - val_loss: 1.6103
Epoch 6/100
106/106 ━━━━━━━━━━━━━━━━━━━━ 5s 47ms/step - accuracy: 0.2037 - loss: 1.6089 - val_accuracy: 0.1919 - val_loss: 1.6102
Epoch 7/100
106/106 ━━━━━━━━━━━━━━━━━━━━ 5s 48ms/step - accuracy: 0.2009 - loss: 1.6088 - val_accuracy: 0.1942 - val_loss: 1.6102
Epoch 8/100
106/106 ━━━━━━━━━━━━━━━━━━━━ 5s 47ms/step - accuracy: 0.2081 - loss: 1.6083 - 

Test accuracy: 0.20779220759868622


In [16]:

# Save the tokenizer
with open('tokenizer.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)

# Save the label encoder
with open('label_encoder.pkl', 'wb') as f:
    pickle.dump(label_encoder, f)